TaxiBJ: InFlow/OutFlow, Meteorology and Holiday ai Beijing
==========================================================

**Proposed by the following paper:**

`Junbo Zhang, Yu Zheng, Dekang Qi. Deep Spatio-Temporal Residual Networks for Citywide Crowd Flows Prediction. In AAAI 2017. `

**TaxiBJ** consists of the following **6** files:

* BJ16_M32x32_T30_InOut.h5
* BJ15_M32x32_T30_InOut.h5
* BJ14_M32x32_T30_InOut.h5
* BJ13_M32x32_T30_InOut.h5
* BJ_Meteorology.h5
* BJ_Holiday.txt

where the first four files are *crowd flows* in Beijing from the year 2013 to 2016, `BJ_Meteorology.h5` is the Meteorological data, `BJ_Holiday.txt` includes the holidays (and adjacent weekends) of Beijing. 

The taxicab GPS data and meteorology data in Beijing from four time intervals: 

- 1st Jul. 2013 - 30th Otc. 2013, 
- 1st Mar. 2014 - 30th Jun. 2014, 
- 1st Mar. 2015 - 30th Jun. 2015, 
- 1st Nov. 2015 - 10th Apr. 2016. 

The Beijing city map is divided into **32 × 32 grids** and the time interval of the flow data is **30 minutes**. 

## `1. View dataset`
### 1.1 Flows of Crowds

File names: `BJ[YEAR]_M32x32_T30_InOut.h5`, where

* YEAR: one of {13, 14, 15, 16}
* M32x32: the Beijing city is divided into a 32 x 32 grid map
* T30: timeslot (a.k.a. time interval) is equal to 30 minites, meaning there are 48 timeslots in a day
* InOut: Inflow/Outflow are defined in the following paper 

Each `*.h5` file has two following subsets:

* `date`: a list of timeslots, which is associated the **data**. 
* `data`: a 4D tensor of shape (number_of_timeslots, 2, 32, 32), of which `data[i]` is a 3D tensor of shape (2, 32, 32) at the timeslot `date[i]`, `data[i][0]` is a `32x32` inflow matrix and `data[i][1]` is a `32x32` outflow matrix. 

In [2]:
# view the data
import h5py
import os

DATAPATH = 'f:/sosa_data/TaxiBJ/'

for i in range(13,17):
    f = h5py.File(os.path.join(DATAPATH, 'BJ{}_M32x32_T30_InOut.h5'.format(str(i))), 'r')
    print(f'BJ20{str(i)}:')
    for k in f.keys():
        print(k, f[k].shape)
    f.close()

BJ2013:
data (4888, 2, 32, 32)
date (4888,)
BJ2014:
data (4780, 2, 32, 32)
date (4780,)
BJ2015:
data (5596, 2, 32, 32)
date (5596,)
BJ2016:
data (7220, 2, 32, 32)
date (7220,)


In [25]:
import time
import h5py

def inspect_flow(file_flow, interval=30, T=48):
    '''print info of file)flow'''
    with h5py.File(file_flow) as f:
        # get the first and the last timeslot
        start = f['date'][0]
        end = f['date'][-1]

        # convert timeslot to struct_time and format string 'YYYY-MM-DD'
        year, month, day = map(int, [start[:4], start[4:6], start[6:8]])
        date_start = time.strptime('%04i-%02i-%02i' % (year, month, day), '%Y-%m-%d')
        date_start_str = time.strftime('%Y-%m-%d', date_start)

        year, month, day = map(int, [end[:4], end[4:6], end[6:8]])
        date_end = time.strptime('%04i-%02i-%02i' % (year, month, day), '%Y-%m-%d')
        date_end_str = time.strftime('%Y-%m-%d', date_end)

        # compute the total number of timeslots and days
        num_timeslots = (time.mktime(date_end) - time.mktime(date_start)) / (interval * 60) + T
        num_days = int(num_timeslots / T)

        max_flow = f['data'][:].max()
        min_flow = f['data'][:].min()

        info = '*-' * 10 + 'flow info' + '*-' * 10 + '\n' + \
            'data shape: %s\n' % str(f['data'].shape) + \
            '#days: %i, from %s to %s\n' % (num_days, date_start_str, date_end_str) + \
            '#timeslots(due): %i, #timeslots(available): %i\n' % (int(num_timeslots), f['date'].shape[0]) + \
            'missing ratio of timeslots: %.1f%%\n' % ((1. - float(f['date'].shape[0]) / num_timeslots) * 100) + \
            f'max_flow: {max_flow:.3f}, min_flow: {min_flow:.3f}\n' 

        print(info)


In [26]:
inspect_flow('f:/sosa_data/TaxiBJ/BJ13_M32x32_T30_InOut.h5')
inspect_flow('f:/sosa_data/TaxiBJ/BJ14_M32x32_T30_InOut.h5')
inspect_flow('f:/sosa_data/TaxiBJ/BJ15_M32x32_T30_InOut.h5')
inspect_flow('f:/sosa_data/TaxiBJ/BJ16_M32x32_T30_InOut.h5')

*-*-*-*-*-*-*-*-*-*-flow info*-*-*-*-*-*-*-*-*-*-
data shape: (4888, 2, 32, 32)
#days: 121, from 2013-07-01 to 2013-10-29
#timeslots(due): 5808, #timeslots(available): 4888
missing ratio of timeslots: 15.8%
max_flow: 1230.000, min_flow: 0.000

*-*-*-*-*-*-*-*-*-*-flow info*-*-*-*-*-*-*-*-*-*-
data shape: (4780, 2, 32, 32)
#days: 119, from 2014-03-01 to 2014-06-27
#timeslots(due): 5712, #timeslots(available): 4780
missing ratio of timeslots: 16.3%
max_flow: 1292.000, min_flow: 0.000

*-*-*-*-*-*-*-*-*-*-flow info*-*-*-*-*-*-*-*-*-*-
data shape: (5596, 2, 32, 32)
#days: 122, from 2015-03-01 to 2015-06-30
#timeslots(due): 5856, #timeslots(available): 5596
missing ratio of timeslots: 4.4%
max_flow: 1274.000, min_flow: 0.000

*-*-*-*-*-*-*-*-*-*-flow info*-*-*-*-*-*-*-*-*-*-
data shape: (7220, 2, 32, 32)
#days: 162, from 2015-11-01 to 2016-04-10
#timeslots(due): 7776, #timeslots(available): 7220
missing ratio of timeslots: 7.2%
max_flow: 1250.000, min_flow: 0.000



### 1.2 Meteorology

File name: `BJ_Meteorology.h5`, which has four following subsets:

* `Temperature`: a list of continuous value, of which the $i^{th}$ value is `temperature` at the timeslot `date[i]`.
* `Weather`: a 2D matrix, each of which is a one-hot vector (`dim=17`). 
* `WindSpeed`: a list of continuous value, of which the $i^{th}$ value is `wind speed` at the timeslot `date[i]`. 
* `date`: a list of timeslots, which is associated the following kinds of data. 

In [23]:
import time

def inspect_time(timeslots, interval=30, T=48):
    start = timeslots[0]
    end = timeslots[-1]
    year, month, day = map(int, [start[:4], start[4:6], start[6:8]])
    start_date = time.strptime(f'{year:04}-{month:02}-{day:02}', '%Y-%m-%d')
    start_date_str = time.strftime('%Y-%m-%d', start_date)
    year, month, day = map(int, [end[:4], end[4:6], end[6:8]])
    end_date = time.strptime(f'{year:04}-{month:02}-{day:02}', '%Y-%m-%d')
    end_date_str = time.strftime('%Y-%m-%d', end_date)

    num_timelots = int((time.mktime(end_date) - time.mktime(start_date)) / (interval * 60) + T)
    num_days = int(num_timelots / T)

    info = f'start date: {start_date_str}, end date: {end_date_str}\n' + \
        f'#days(due): {num_days}\n' + \
        f'#slots(due): {num_timelots}, #slots(available): {timeslots.shape[0]}\n'
    
    print(info)

In [24]:
import os
import h5py

DATAPATH = 'f:/sosa_data/TaxiBJ/'

f_meteorology  = h5py.File(os.path.join(DATAPATH, 'BJ_Meteorology.h5'), 'r')
for k in f_meteorology:
    print(k, f_meteorology[k].shape)
inspect_time(f_meteorology['date'])
f_meteorology.close()

Temperature (59006,)
Weather (59006, 17)
WindSpeed (59006,)
date (59006,)
start date: 2013-02-01, end date: 2016-06-13
#days(due): 1229
#slots(due): 58992, #slots(available): 59006



### 1.3 Holiday

File name: `BJ_Holiday.txt`, which includes a list of the holidays (and adjacent weekends) of Beijing. 

Each line a holiday with the data format [yyyy][mm][dd]. For example, `20150601` is `June 1st, 2015`. 

In [14]:
import os

f_holiday = open(os.path.join(DATAPATH, 'BJ_Holiday.txt'), 'r')
holidays = f_holiday.read().split()
print(len(holidays), holidays[0])
f_holiday.close()

106 20130101


## ` 2. Load data`

- `data_flow`: ndarray, (22484, 2, 32, 32)
- `timeslot_flow`: ndarray, (22484,)
- `meteorology_data`: ndarray, (22484, 19)
- `is_holiday`: ndarray, (22484,)

In [22]:
import os
import h5py
import numpy as np

DATAPATH = 'f:/sosa_data/TaxiBJ/'

def load_flow(file_path):
    '''load inflow and outflow gride data from year 2013 to 2016

    Input:
    - file_path: Path of the flow data file, a string like '*/TaxiBJ/'

    Output:
    - data_flow: ndarray of all flow data with shape (22484, 2, 32, 32)
    - timeslot_flow: ndarray of all timeslots with shape (22484,)
    '''
    data_flow = []
    timeslot_flow = []
    for year in range(13, 17):
        file_name = os.path.join(file_path, f'BJ{str(year)}_M32x32_T30_InOut.h5')
        f_flow = h5py.File(file_name, 'r')
        data = f_flow['data'][()]
        timeslot = f_flow['date'][()]
        data_flow.append(data)
        timeslot_flow.append(timeslot)
        f_flow.close()
    data_flow = np.vstack(data_flow)
    timeslot_flow = np.concatenate(timeslot_flow,axis=0)
    return data_flow, timeslot_flow


# data_flow, timeslot_flow = load_flow(DATAPATH)

In [26]:
def load_meteorology(timeslots, file_path):
    '''load file 'BJ_Meteorology.h5'

    Input:
    - timeslots: ndarray of timeslots with shape (#timeslots,) , (22484,) if load all data
    - file_path: Path of the meteorology data file, a string like '*/TaxiBJ/'

    Output:
    - meteorology_data:  ndarray with shape (#timeslots, 19), (22484, 19) if load all data
    '''
    f_meteorology = h5py.File(os.path.join(file_path, 'BJ_Meteorology.h5'), 'r')
    Timeslot = f_meteorology['date'][()]
    WindSpeed = f_meteorology['WindSpeed'][()]
    Weather = f_meteorology['Weather'][()]
    Temperature = f_meteorology['Temperature'][()]
    f_meteorology.close()

    M = dict()  # map timeslot to index
    for i, slot in enumerate(Timeslot):
        M[slot] = i

    WS = []  # WindSpeed
    WR = []  # Weather
    TE = []  # Temperature
    for slot in timeslots:
        predicted_id = M[slot]
        current_id = predicted_id - 1
        WS.append(WindSpeed[current_id])
        WR.append(Weather[current_id])
        TE.append(Temperature[current_id])

    WS = np.asarray(WS) # (?, 1)
    WR = np.asarray(WR) # (?, 17)
    TE = np.asarray(TE) # (?, 1)

    # 0-1 scale
    # WS = 1. * (WS - WS.min()) / (WS.max() - WS.min())
    # TE = 1. * (TE - TE.min()) / (TE.max() - TE.min())

    meteorology_data = np.hstack([WR, WS[:, None], TE[:, None]])    # concatenate all these attributes

    return meteorology_data

# meteorology_data = load_meteorology(timeslot_flow, DATAPATH)

In [18]:
def load_holiday(timeslots, file_path):
    '''load file 'BJ_Holiday.txt' 
    
    Input:
    - timeslots: ndarray of timeslots with shape (22484,) if load all data
    - file_path: Path of the holiday data file, a string like '*/TaxiBJ/'

    Output:
    - is_holiday: ndarray of {0, 1} with shape (#timeslots, 1), 1 indicating whether current timeslot is holiday
    '''
    f_holiday = open(os.path.join(file_path, 'BJ_Holiday.txt'), 'r')
    holidays = f_holiday.read().split()
    holidays = set([h.strip() for h in holidays])   # remove the leading and trailing whitespace of the string
    is_holiday = np.zeros(len(timeslots))
    for i, slot in enumerate(timeslots):
        s = bytes.decode(slot[:8])  # Decode the bytes to string
        if s in holidays:
            is_holiday[i] = 1
    return is_holiday[:, None]


# is_holiday = load_holiday(timeslot_flow, DATAPATH)

## `3. Preprocessing`

### 3.1 missing data process

Remove the data of days which do not have 48 timestamps, after removed:

data_list:
- `data_flow`: ndarray, (21360, 2, 32, 32)
- `meteorology_data`: ndarray, (21360, 19)
- `is_holiday`: ndarray, (21360, 1)
  
timestamps:
- `timestamps`: ndarray, (21360,)

In [97]:
def remove_incomplete_days(data, timestamps, T=48):
    '''remove a certain day which does not have 48 timestamps
    '''
    days = []  # available days: some day only contain some seqs
    days_incomplete = []
    i = 0
    while i < len(timestamps):
        if int(timestamps[i][8:]) != 1:
            i += 1
        elif i+T-1 < len(timestamps) and int(timestamps[i+T-1][8:]) == T:
            days.append(timestamps[i][:8])
            i += T
        else:
            days_incomplete.append(timestamps[i][:8])
            i += 1
    days = set(days)
    idx = []
    for i, t in enumerate(timestamps):
        if t[:8] in days:
            idx.append(i)

    data = data[idx]
    timestamps = [timestamps[i] for i in idx]
    timestamps = np.asarray(timestamps)

    return data, timestamps, days_incomplete


### 3.2 get time position feature



In [1]:
import numpy as np
import time
import sys
sys.path.append("E:\\AdaSTTP")

from fuzzy_utils import triangular, asymmetric_gaussian, get_triangular_params, get_guassian_params

def time_of_day(timestamps, fuzzy=None, time_position_list=[13, 18, 29, 34, 38, 45], T=48):
    '''the time position in one day

    Input:
    - timestampes: A list or ndarray of timeslots with the format 'YYYYMMDDNN', 'NN' indicates the NN^th timeslot in the day 'YYYYMMDD'
    - fuzzy: None, 'triangular' or 'guassian', default None. 
    - time_position_list: None or a list of end time position of each period.

    Output:
    - time_position: A list of float number indicating which time position period the current slot belongs to.
    '''

    time_of_day = [int(t[8:10]) for t in timestamps]    # integer [1, T]
    # number of positions in one day
    num_positions = len(time_position_list)
    time_position = []
    if fuzzy == None:
        for i in time_of_day:
            p = [0. for _ in range(num_positions)]
            for j in range(num_positions):
                if time_position_list[j] >= i:
                    p[j] = 1.
                    break
                elif time_position_list[-1] < i:
                    p[0] = 1.
            
            time_position.append(p)

    elif fuzzy == 'triangular':
        a_list, b_list, c_list = get_triangular_params(time_position_list)
        
    elif fuzzy == 'guassian':
        a_list, b_list, c_list = get_guassian_params(time_position_list)

    for i in time_of_day:
        p = [0. for _ in range(num_positions)]
        for j in range(num_positions):
            a, b, c = a_list[j], b_list[j], c_list[j]
            p[j] = triangular(i, a, b, c) if fuzzy == 'triangular' else asymmetric_gaussian(i, a, b, c)
    time_position.append(p)

    return np.asarray(time_position)


def day_of_week(timestamps):
    '''day of week, is weekday or not

    Input:
    - timestampes: A list or ndarray of timeslots with the format 'YYYYMMDDNN', 'NN' indicates the NN^th timeslot in the day 'YYYYMMDD'

    Output:
    - week_position[:7]: one-hot encoding indicating the day of week
    - week_position[7]: 0--weekend, 1--weekday
    '''
    day_of_week = [time.strptime(str(
        t[:8], encoding='utf-8'), '%Y%m%d').tm_wday for t in timestamps]  # tm_wday(): day of week, range [0, 6], Monday is 0
    week_position = []
    for i in day_of_week:
        v = [0 for _ in range(7)]
        v[i] = 1
        if i >= 5:
            v.append(0)  # weekend
        else:
            v.append(1)  # weekday
        week_position.append(v)

    return np.asarray(week_position)

In [2]:
def load_time_position(timestamps):
    '''time of day, day of week, and is weekday or not

    Input:
    - timestampes: A list or ndarray of timeslots with the format 'YYYYMMDDNN', 'NN' indicates the NN^th timeslot in the day 'YYYYMMDD'

    Output:
    - time_meta[:7]: one-hot encoding indicating the day of week
    - time_meta[7]: 0--weekend, 1--weekday
    - time_meta[8:]: membership degree of each time period 
    '''
    week_position = day_of_week(timestamps)
    time_position = time_of_day(timestamps)
    time_meta = week_position.concatenate(time_position, 0)
    
    return time_meta

### 3.2 merge feature

- data: (#timestamps, 2, 32, 32)
- timestamp: (#timestamps,)
- extra_feature: (#timestamps, 28 + #time_periods):
  - [:, :17] weather
  - [:, 17] wind speed
  - [:, 18] temperature
  - [:, 19] is_holiday
  - [:, 20:27] day of week
  - [:, 27] is_workday
  - [:, 28:] time of day

In [28]:
def get_merge_feature(out_dir, use_meteorology=True, use_time_meta=True, use_holiday=True):
    """
    load flow data from 2013 to 2016, as well as corresponding extra features
    """
    
    data_flow, timeslot_flow = load_flow(DATAPATH)  
    # remove data of days with missing values
    data_flow_rm, timeslot_flow_rm = remove_incomplete_days(data_flow, timeslot_flow)
    data_flow_rm[data_flow_rm<0] = 0.

    extra_feature = []
    if use_meteorology:
        meteorology_feature = load_meteorology(timeslot_flow_rm, DATAPATH)
        extra_feature.append(meteorology_feature)
        # print('meteorol feature: ', meteorology_feature.shape) -- meteorol feature:  (21360, 19)
    if use_holiday:
        holiday_feature = load_holiday(timeslot_flow_rm, DATAPATH)
        extra_feature.append(holiday_feature)
        # print('holiday feature:', holiday_feature.shape) -- holiday feature: (21360, 1)
    if use_time_meta:
        time_meta_feature = load_time_position(timeslot_flow_rm)
        extra_feature.append(time_meta_feature)
        # print('time feature:', time_meta_feature.shape)  -- time feature: (21360, 8)


    extra_feature = np.hstack(extra_feature) if len(extra_feature) > 0 else np.asarray(extra_feature)
    # print('mete feature: ', meta_feature.shape) -- mete feature:  (21360, 28)

    np.savez(out_dir, data=data_flow_rm, timestamp=timeslot_flow_rm, extra_feature=extra_feature)
    

In [31]:
get_merge_feature(os.path.join(DATAPATH, 'taxibj.npz'))

### 3.3 train/val/test division

The total number of timestamps is 21360, the number of timestamps in one day is 48 --> # days = 21360 / 48 = 445
- train_data: the data of first `389` days, timestamps[:18672]
  - train_data -- (18672, 2, 32, 32)
  - train_slots -- (18672,)
  - train_meta_feature -- (18672, 28 + #time_periods)
- val_data: the data of the following `28` days, 28, timestamps[18672:21016]
  - val_data -- (1344, 2, 32, 32)
  - val_slots -- (1344,)
  - val_meta_feature -- (1344, 28 + #time_periods)
- test_date: the data of the last `28` days, timestamps[-1344:]
  - test_data -- (1344, 2, 32, 32)
  - test_slots -- (1344,)
  - test_meta_feature -- (1344, 28 + #time_periods)

In [21]:
import numpy as np

def split_dataset(file_path, train_days=389, val_days=28, test_days=28, T=48, use_meta_feature=True):
    
    data_all = np.load(open(file_path, 'rb'))

    train_data = data_all['data'][:train_days * T]
    train_slots = data_all['timestamp'][:train_days * T]
    val_data = data_all['data'][train_days * T : (train_days + val_days) * T]
    val_slots = data_all['timestamp'][train_days * T : (train_days + val_days) * T]
    test_data = data_all['data'][-test_days * T:]
    test_slots = data_all['timestamp'][-test_days * T:]
    if use_meta_feature:
        train_meta_feature = data_all['meta_feature'][:train_days * T]
        val_meta_feature =  data_all['meta_feature'][train_days * T : (train_days + val_days) * T]
        test_meta_feature = data_all['meta_feature'][-test_days * T:]
    
    data_split = {
            'train_data': train_data,
            'train_slots': train_slots,
            'val_data': val_data,
            'val_slots': val_slots,
            'test_data': test_data,
            'test_slots': test_slots,
            'train_meta_feature': train_meta_feature if use_meta_feature else None,
            'val_meta_feature': val_meta_feature if use_meta_feature else None,
            'test_meta_feature': test_meta_feature if use_meta_feature else None
            }

    data_all.close()
    return data_split


In [105]:
data_split = split_dataset('f:/sosa_data/TaxiBJ/taxibj.npz')

np.savez('f:/sosa_data/TaxiBJ/taxibj_split.npz',
        train_data = data_split['train_data'],
        train_slots = data_split['train_slots'],
        val_data = data_split['val_data'],
        val_slots = data_split['val_slots'],
        test_data = data_split['test_data'],
        test_slots = data_split['test_slots'],
        train_meta_feature = data_split['train_meta_feature'],
        val_meta_feature = data_split['val_meta_feature'],
        test_meta_feature = data_split['test_meta_feature'])

## 4. visual data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig_flow = plt.figure(num=1, figsize=(25, 18))

ax1 = fig_flow.add_subplot(311)
# ax1.plot(data_split['test_data'][-48:, 0, 0, 0], [i for i in range(48)])
ax1.plot(data_split['test_data'][-48*14:, 0, 12, 12], 'r-', label='location (12, 12)') 
ax1.plot(data_split['test_data'][-48*14:, 0, 12, 13], 'g-', label='location (12, 13)') 
ax1.plot(data_split['test_data'][-48*14:, 0, 13, 12], 'b-', label='location (13, 12)') 
ax1.set_xticks(np.linspace(0, 48*14, 14))
ax1.set_xticklabels(i for i in range(1,15))
ax1.set_title('Inflow data of the last 14 days in test set')
ax1.legend(loc=1, labelspacing=2)


ax2 = fig_flow.add_subplot(312)
ax2.plot(data_split['val_data'][-48*14:, 0, 12, 12], 'r-', label='location (12, 12)') 
ax2.plot(data_split['val_data'][-48*14:, 0, 12, 13], 'g-', label='location (12, 13)') 
ax2.plot(data_split['val_data'][-48*14:, 0, 13, 12], 'b-', label='location (13, 12)') 
ax2.set_xticks(np.linspace(0, 48*14, 14))
ax2.set_xticklabels(i for i in range(1,15))
ax2.set_title('Inflow data of the last 14 days in validation set')
ax2.legend(loc=1, labelspacing=2)


ax3 = fig_flow.add_subplot(313)
ax3.plot(data_split['train_data'][-48*14:, 0, 12, 12], 'r-', label='location (12, 12)') 
ax3.plot(data_split['train_data'][-48*14:, 0, 12, 13], 'g-', label='location (12, 13)') 
ax3.plot(data_split['train_data'][-48*14:, 0, 13, 12], 'b-', label='location (13, 12)') 
ax3.set_xticks(np.linspace(0, 48*14, 14))
ax3.set_xticklabels(i for i in range(1,15))
ax3.set_title('Inflow data of the last 14 days in training set')
ax3.legend(loc=1, labelspacing=2)

plt.show()

In [11]:
print(X.shape)
print(Y.shape)

(0,)
(0,)


In [6]:
a = np.random.randn(4,2,3,3)
b = np.vstack((a[0], a[1]))

In [18]:
cannot import  from partially initialized module (most likely due to a circular import)

In [ ]:
'f:/sosa_data/TaxiBJ/processed/'